In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [74]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [75]:
data=pd.read_csv('final_data_en.csv')
data=data[['text','class']]
data

,text,class
0,RT @maideeeni: I want to say a very big thank ...,anxiety
1,@AisYalcaE @clyrhck Done! I just need 320! Tha...,lonely
2,@SAD_istfied I'm gonna cry /pos,stress
3,RT @JoshOfficial___: I need a consistent fuck ...,anxiety
4,RT @mikegolicjr: glad we're finally talking ab...,normal
...,...,...
39761,Im nervous ash to fly though,anxiety
39762,64E7B6DA :Battle ID\nI need backup!\nLvl 120 G...,lonely
39763,SNOW MUCH FUN writing about our snow days in @...,anxiety
39764,Very time sensitive. I'm trans and my roommate...,anxiety


In [76]:
import numpy as np
import tensorflow as tf
# train=data.loc[ :33802]
# test=data.loc[33803: ]
data=data.sample(frac=1)


data

,text,class
12300,anyone need tbz? i need this kevin https://t....,anxiety
182,"Im not doing okay, no I dont want to talk abou...",anxiety
21523,Still thinking about Mango Recolored I need to...,lonely
24948,RT @moonaquilas: me: i need to fix my sleeping...,anxiety
36360,RT @ninomaeinanis: Finished!!!! &gt; w &lt; )/...,normal
...,...,...
32833,RT @chriscapossela: Were growing in Atlanta! @...,normal
29782,RT @hwngsource: fireworks\n\n191216 ISAC\n# #...,normal
33999,RT @MelBrooks: Such sad newsCloris was insanel...,stress
5480,RT @AOC: The sad thing about disinformation is...,stress


In [77]:
data.dropna(inplace=True)



In [78]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [79]:
data

,text,class
12300,anyone need tbz? i need this kevin https://t....,anxiety
182,"Im not doing okay, no I dont want to talk abou...",anxiety
21523,Still thinking about Mango Recolored I need to...,lonely
24948,RT @moonaquilas: me: i need to fix my sleeping...,anxiety
36360,RT @ninomaeinanis: Finished!!!! &gt; w &lt; )/...,normal
...,...,...
32833,RT @chriscapossela: Were growing in Atlanta! @...,normal
29782,RT @hwngsource: fireworks\n\n191216 ISAC\n# #...,normal
33999,RT @MelBrooks: Such sad newsCloris was insanel...,stress
5480,RT @AOC: The sad thing about disinformation is...,stress


In [80]:
import pandas as pd
import sklearn
import numpy as np
import re
import nltk

import matplotlib.pyplot as plt
from matplotlib import style

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string

# allow display of multiple outputs by running one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [81]:
cachedStopWords = list(esw)
lemmatizer=WordNetLemmatizer()
#ps = PorterStemmer()

def lemmatize_article(sentence):
    sentence = word_tokenize(sentence)
    res = ''
    for word, tag in pos_tag(sentence):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        word = lemmatizer.lemmatize(word, wntag) if wntag else word
        res += word + ' '
    return res


def remove_stop_words(sentence):
    return ' '.join([word for word in sentence.split() if word not in cachedStopWords])
    
def remove_short(sentence):
    return ' '.join([word for word in sentence.split() if len(word) >= 3])
    
def remove_digits(sentence):
    return ' '.join([i for i in sentence.split() if not i.isdigit()])
    
def preprocess(all_texts):
    all_texts = list(map(lambda x: x.lower(), all_texts))
    all_texts = list(map(lambda x: x.translate(str.maketrans('', '', string.punctuation)), all_texts))
    all_texts = list(map(lambda x: lemmatize_article(x), all_texts))
    all_texts = list(map(lambda x: x.strip(), all_texts))
    all_texts = list(map(lambda x: remove_stop_words(x), all_texts))
    all_texts = list(map(lambda x: remove_short(x), all_texts))
    all_texts = list(map(lambda x: remove_digits(x), all_texts))
    return all_texts

In [82]:
data['text'] = data['text'].str.replace(r'\s*@\w+', '', regex = True)
data['text'] = data['text'].str.replace(r'\s*\B@\w+', '', regex = True)
data['text'] = data['text'].str.replace(r'\s*@\S+', '', regex = True)
data['text'] = data['text'].str.replace(r'\s*@\S+\b', '', regex = True)
data['text'] = data['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data


,text,class
12300,anyone need tbz? i need this kevin,anxiety
182,"Im not doing okay, no I dont want to talk abou...",anxiety
21523,Still thinking about Mango Recolored I need to...,lonely
24948,RT: me: i need to fix my sleeping schedule \n\...,anxiety
36360,RT: Finished!!!! &gt; w &lt; )// # #,normal
...,...,...
32833,RT: Were growing in Atlanta! just shared plans...,normal
29782,RT: fireworks\n\n191216 ISAC\n# # #hyunjin,normal
33999,RT: Such sad newsCloris was insanely talented....,stress
5480,RT: The sad thing about disinformation is that...,stress


In [83]:
data['text'] = preprocess(data['text'])
data

,text,class
12300,need tbz need kevin,anxiety
182,okay dont want talk emotionally available prom...,anxiety
21523,think mango recolored need draw feel vibe emot...,lonely
24948,need fix sleeping schedule 3am,anxiety
36360,finish,normal
...,...,...
32833,grow atlanta just share plan make multiple inv...,normal
29782,fireworks isac hyunjin,normal
33999,sad newscloris insanely talented make laugh dr...,stress
5480,sad thing disinformation truth come damage people,stress


In [84]:
data = pd.DataFrame(data)
data.head()

,text,class
12300,need tbz need kevin,anxiety
182,okay dont want talk emotionally available prom...,anxiety
21523,think mango recolored need draw feel vibe emot...,lonely
24948,need fix sleeping schedule 3am,anxiety
36360,finish,normal


In [85]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier

In [86]:
data= data[(data["text"] != "") & (data["text"] != "null")]


In [87]:
data.shape

(39555, 2)

In [88]:
data['class'].value_counts()

anxiety    12573
normal      9844
stress      8594
lonely      8544
Name: class, dtype: int64

In [89]:
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [90]:
# Encoding the label column
data['class']=data['class'].map({'anxiety':1,'normal':0,'stress':0,'lonely':0})
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split (data['text'], data['class'] , test_size=0.2)

In [111]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train)

In [112]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


In [113]:
for i, v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]), len(v))

Streaming output truncated to the last 5000 lines.
87 87
4 4
42 42
18 18
51 51
34 34
6 6
102 102
119 119
23 23
31 31
137 137
6 6
50 50
38 38
87 87
61 61
76 76
37 37
29 29
38 38
39 39
139 139
61 61
73 73
3 3
12 12
26 26
28 28
70 70
67 67
56 56
51 51
24 24
31 31
21 21
28 28
130 130
55 55
70 70
13 13
14 14
17 17
138 138
4 4
119 119
80 80
19 19
44 44
9 9
47 47
8 8
73 73
48 48
62 62
53 53
45 45
38 38
72 72
81 81
31 31
53 53
27 27
12 12
18 18
103 103
103 103
56 56
56 56
56 56
16 16
31 31
70 70
124 124
55 55
57 57
87 87
74 74
25 25
48 48
48 48
63 63
113 113
140 140
64 64
18 18
3 3
71 71
28 28
60 60
64 64
55 55
98 98
9 9
9 9
42 42
24 24
66 66
71 71
58 58
20 20
5 5
58 58
48 48
139 139
53 53
168 168
53 53
69 69
115 115
4 4
89 89
59 59
18 18
82 82
14 14
50 50
124 124
13 13
65 65
13 13
103 103
23 23
168 168
90 90
146 146
115 115
44 44
52 52
125 125
106 106
28 28
6 6
19 19
99 99
58 58
6 6
68 68
21 21
47 47
66 66
32 32
30 30
154 154
4 4
36 36
8 8
59 59
95 95
117 117
48 48
130 130
56 56
150 150
59 59

In [114]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [115]:
for i, v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]), len(v))

Streaming output truncated to the last 5000 lines.
87 100
4 100
42 100
18 100
51 100
34 100
6 100
102 100
119 100
23 100
31 100
137 100
6 100
50 100
38 100
87 100
61 100
76 100
37 100
29 100
38 100
39 100
139 100
61 100
73 100
3 100
12 100
26 100
28 100
70 100
67 100
56 100
51 100
24 100
31 100
21 100
28 100
130 100
55 100
70 100
13 100
14 100
17 100
138 100
4 100
119 100
80 100
19 100
44 100
9 100
47 100
8 100
73 100
48 100
62 100
53 100
45 100
38 100
72 100
81 100
31 100
53 100
27 100
12 100
18 100
103 100
103 100
56 100
56 100
56 100
16 100
31 100
70 100
124 100
55 100
57 100
87 100
74 100
25 100
48 100
48 100
63 100
113 100
140 100
64 100
18 100
3 100
71 100
28 100
60 100
64 100
55 100
98 100
9 100
9 100
42 100
24 100
66 100
71 100
58 100
20 100
5 100
58 100
48 100
139 100
53 100
168 100
53 100
69 100
115 100
4 100
89 100
59 100
18 100
82 100
14 100
50 100
124 100
13 100
65 100
13 100
103 100
23 100
168 100
90 100
146 100
115 100
44 100
52 100
125 100
106 100
28 100
6 100
19 100
99

In [116]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [117]:
y_pred = rf_model.predict(X_test_vect_avg)

In [118]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 0.796 / Recall: 0.796 / Accuracy: 0.796
